# Machine Learning Model

Machine Learning Model for user sign in - this takes in a list of keystroke patterns of the attempted logins from the user. It trains the model each attempt and returns a confidence score based on the new provided keystroke pattern vs what the system would expect.

In [ ]:
#This work will be annotated using the Better Comments Extension
#! This is for notes and for things that need to be worked on, mainly error handling
#? This is for questioning if this is finished or needs adding/reworking
#*This text will highlight, this will mainly be for notes
#//This should cross out any lines of code that are no longer needed or used during testing.
#TODO This is for annotating work that needs to be implemented.

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import sys
import json

In [ ]:
#* Data Preperation
#data = #This will be the list sent over from the C# project
#expected = #This will be the bit value from the table
if len(sys.argv) > 1:
    json_list = sys.argv[1]
    object_list = json.loads(json_list)
    
    #TODO Begin Machine Learning
    #//confidence_value = your_machine_learning_function(object_list)
    
    print(confidence_value)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data)

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)
